# Porto's Taxis Data Cleaning

### Imports

In [1]:
import ast
from datetime import timedelta

import pandas as pd
from pandas import Timestamp

### Load Data

In [2]:
df = pd.read_csv("data/train.csv.zip", compression="zip")

In [3]:
df

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
...,...,...,...,...,...,...,...,...,...
1710665,1404171463620000698,C,NaN,NaN,20000698,1404171463,A,False,"[[-8.612469,41.14602],[-8.612487,41.145993],[-..."
1710666,1404171367620000670,C,NaN,NaN,20000670,1404171367,A,False,"[[-8.610138,41.140845],[-8.610174,41.140935],[..."
1710667,1388745716620000264,C,NaN,NaN,20000264,1388745716,A,False,[]
1710668,1404141826620000248,B,NaN,12.0,20000248,1404141826,A,False,"[[-8.630712,41.154885],[-8.63073,41.154813],[-..."


## Missing Values

In [4]:
# How many rows have missing values?
df["MISSING_DATA"].value_counts()

MISSING_DATA
False    1710660
True          10
Name: count, dtype: int64

In [5]:
# Remove rows with missing data
df = df[df["MISSING_DATA"] == False].drop(columns=["MISSING_DATA"])

In [6]:
# What rows have 0 coordinates?
df[df["POLYLINE"] == "[]"]

# Probably cancelled trips before starting

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,POLYLINE
762,1372665673620000353,C,NaN,NaN,20000353,1372665673,A,[]
1161,1372669158620000562,C,NaN,NaN,20000562,1372669158,A,[]
1459,1372665875620000496,C,NaN,NaN,20000496,1372665875,A,[]
1677,1372667320620000288,C,NaN,NaN,20000288,1372667320,A,[]
1719,1372676112620000600,C,NaN,NaN,20000600,1372676112,A,[]
...,...,...,...,...,...,...,...,...
1710547,1403120685620000226,A,26552.0,NaN,20000226,1403120685,A,[]
1710643,1386603894620000970,C,NaN,NaN,20000970,1386603894,A,[]
1710658,1373986578620000931,A,2002.0,NaN,20000931,1373986578,A,[]
1710664,1390403767620000911,C,NaN,NaN,20000911,1390403767,A,[]


In [7]:
# Remove rows with empty coordinates
df = df[df["POLYLINE"] != "[]"]

### Sanity Check for the `ORIGIN_CALL` and `ORIGIN_STAND` Columns

In [8]:
# There shouldn't be any rows with origin_call set that are not of type A
df[["ORIGIN_CALL", "CALL_TYPE"]].groupby("CALL_TYPE").count()

,ORIGIN_CALL
CALL_TYPE,
A,364344
B,0
C,0


In [9]:
# There shouldn't be any rows with origin_stand set that are not of type B
df[["ORIGIN_STAND", "CALL_TYPE"]].groupby("CALL_TYPE").count()

,ORIGIN_STAND
CALL_TYPE,
A,0
B,804773
C,0


In [10]:
# There's some duplicate data in the dataset
df[df.duplicated(keep=False)]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,POLYLINE
998727,1391416612620000196,C,NaN,NaN,20000196,1391416612,A,"[[-8.555265,41.180472]]"
998728,1391416612620000196,C,NaN,NaN,20000196,1391416612,A,"[[-8.555265,41.180472]]"
1304114,1397172149620000454,C,NaN,NaN,20000454,1397172149,A,"[[-8.618391,41.162067],[-8.6184,41.161446],[-8..."
1304115,1397172149620000454,C,NaN,NaN,20000454,1397172149,A,"[[-8.618391,41.162067],[-8.6184,41.161446],[-8..."


In [11]:
# Let's remove the duplicates
df = df.drop_duplicates()

In [12]:
# There are also some rows with the same TRIP_ID
df[df.duplicated(subset=["TRIP_ID"], keep=False)]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,POLYLINE
3519,1372702836620000080,B,NaN,16.0,20000080,1372702836,A,"[[-8.635275,41.193729],[-8.634519,41.192901]]"
3720,1372702836620000080,C,NaN,NaN,20000080,1372702836,A,"[[-8.62776,41.17554],[-8.62776,41.17554],[-8.6..."
33228,1373210896620000598,B,NaN,57.0,20000598,1373210896,A,"[[-8.610885,41.145525]]"
33350,1373210896620000598,C,NaN,NaN,20000598,1373210896,A,"[[-8.610813,41.14548],[-8.610732,41.145579],[-..."
51623,1373551710620000014,B,NaN,49.0,20000014,1373551710,A,"[[-8.587278,41.147325],[-8.587278,41.147325]]"
...,...,...,...,...,...,...,...,...
1660300,1403323491620000600,C,NaN,NaN,20000600,1403323491,A,"[[-8.602533,41.155686],[-8.602191,41.15646],[-..."
1684526,1403716904620000337,C,NaN,NaN,20000337,1403716904,A,"[[-8.654913,41.182821],[-8.656623,41.182317],[..."
1684739,1403716904620000337,C,NaN,NaN,20000337,1403716904,A,"[[-8.654913,41.182821],[-8.656623,41.182317],[..."
1705646,1404093316620000307,B,NaN,57.0,20000307,1404093316,A,"[[-8.610831,41.145741]]"


In [13]:
# They're not that many, and we don't know which one to keep, so let's just remove them
df = df.drop_duplicates(subset=["TRIP_ID"], keep=False)

## Preprocessing

In [14]:
df = df.convert_dtypes()

df["TIMESTAMP"] = df["TIMESTAMP"].map(lambda x: Timestamp(x, unit="s", tz="Portugal"))
df["TIMESTAMP"] = df["TIMESTAMP"].dt.tz_convert("UTC").dt.tz_localize(None)

In [15]:
# DAYTYPE: (char) It identifies the daytype of the trip's start. It assumes one of three possible values:
#   - 'B' if this trip started on a holiday or any other special day (i.e. extending holidays, floating holidays, etc.);
#   - 'C' if the trip started on a day before a type-B day;
#   - 'A' otherwise (i.e. a normal day, workday or weekend)

# The field DAY_TYPE was not computed correctly (all values are 'A'). Let's fix this.

holidays = [
    Timestamp("2013-01-01"),
    Timestamp("2013-03-29"),
    Timestamp("2013-03-31"),
    Timestamp("2013-04-25"),
    Timestamp("2013-05-01"),
    Timestamp("2013-05-30"),
    Timestamp("2013-06-10"),
    Timestamp("2013-08-15"),
    Timestamp("2013-08-16"),
    Timestamp("2013-10-05"),
    Timestamp("2013-11-01"),
    Timestamp("2013-12-01"),
    Timestamp("2013-12-08"),
    Timestamp("2013-12-25"),
    Timestamp("2014-01-01"),
    Timestamp("2014-04-18"),
    Timestamp("2014-04-25"),
    Timestamp("2014-05-01"),
    Timestamp("2014-06-10"),
    Timestamp("2014-06-19"),
    Timestamp("2014-08-15"),
    Timestamp("2014-10-05"),
    Timestamp("2014-11-01"),
    Timestamp("2014-12-01"),
    Timestamp("2014-12-08"),
    Timestamp("2014-12-25")
]

# How many trips that started on a 25th of December?
df[(df["TIMESTAMP"].dt.month == 12) & (df["TIMESTAMP"].dt.day == 25)]


def get_day_type(ts: Timestamp):
    date = ts.normalize()
    if date in holidays:
        return 'B'
    elif (date + pd.Timedelta(days=1)) in holidays:
        return 'C'
    else:
        return 'A'


df["DAY_TYPE"] = df["TIMESTAMP"].map(get_day_type)


In [16]:
df["POLYLINE"] = df["POLYLINE"].map(ast.literal_eval)
df["LONGITUDE"] = df["POLYLINE"].map(lambda coords: [c[0] for c in coords])
df["LATITUDE"] = df["POLYLINE"].map(lambda coords: [c[1] for c in coords])

In [17]:
df["TIMESTAMP"] = df.apply(
    lambda row: [row["TIMESTAMP"] + timedelta(seconds=15 * i) for i in range(len(row["POLYLINE"]))], axis=1)

In [18]:
df["START"] = df["POLYLINE"].map(lambda coords: coords[0])
df["END"] = df["POLYLINE"].map(lambda coords: coords[-1])

In [19]:
# There were some rows with few points but the distance covered was impossible
# Let's remove the rows with less than 5 points (60s of travel or less)
df[df["POLYLINE"].map(len) < 5].count()

TRIP_ID         46099
CALL_TYPE       46099
ORIGIN_CALL      3102
ORIGIN_STAND    10350
TAXI_ID         46099
TIMESTAMP       46099
DAY_TYPE        46099
POLYLINE        46099
LONGITUDE       46099
LATITUDE        46099
START           46099
END             46099
dtype: int64

In [20]:
# How much percentage of the data is this?
df[df["POLYLINE"].map(len) < 5].count() / df.count()
# 2.7%

TRIP_ID         0.027044
CALL_TYPE       0.027044
ORIGIN_CALL     0.008514
ORIGIN_STAND    0.012861
TAXI_ID         0.027044
TIMESTAMP       0.027044
DAY_TYPE        0.027044
POLYLINE        0.027044
LONGITUDE       0.027044
LATITUDE        0.027044
START           0.027044
END             0.027044
dtype: float64

In [21]:
df = df[df["POLYLINE"].map(len) >= 5]

In [22]:
# Save the cleaned data
df.to_parquet("data/train_cleaned.parquet", index=False)